In [ ]:
import autolens as al
# import autolens.plot as aplt
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import random

import os
import time
import datetime

In [34]:
WIDTH = 512
HEIGHT = 512
SIM_NO = 0
NO_OF_IMAGES_TO_GENERATE = 500
PIXEL_SCAlES = 0.01
MIN_RADIUS = .2
MIN_SOURCE_INTENSITY = 1
MIN_LENSER_INTENSITY = .5

CLASS_NAMES = ['Massless', 'Chameleon', 'DeVaucoulers', 'Exponential', 'Gaussian', 'gNFW', 'Isothermal', 'Sersic']

grid = al.Grid2D.uniform(shape_native=(HEIGHT,WIDTH), pixel_scales=PIXEL_SCAlES)

In [ ]:
# print(os.path.exists('sim1/trial.txt'))

In [ ]:
def find_empty_file_starting_with(starter):
    temp = 0
    file_name = ""
    while 1:
        file_name = starter + str(temp) + '.png'
        if not os.path.exists(file_name):
            return file_name
        temp += 1

    # Code should never come here
    return file_name

def generate_ell_comps():
    fac = min(random.random(), .5)
    angle = random.random() * 2 * np.pi

    return (fac * np.sin(2*angle), fac * np.cos(2*angle))

In [35]:
for iter in range(NO_OF_IMAGES_TO_GENERATE):
    tic = time.time()

    # ----CREATING THE SOURCE GALAXY------
    source_centre = (random.random()*HEIGHT*PIXEL_SCAlES/3, random.random()*WIDTH*PIXEL_SCAlES/3)

    # DECIDING THE SOURCE'S BULDGE
    source_buldge_type = random.randint(0,3)
    source_intensity = max(random.random(), MIN_SOURCE_INTENSITY)
    source_ell_comps = generate_ell_comps()
    source_buldge = None
    if source_buldge_type == 0:
        source_buldge = al.lp.Gaussian(
            centre=source_centre,
            ell_comps=source_ell_comps,
            intensity=source_intensity,
            sigma=max(random.random(), MIN_RADIUS)
        )
    elif source_buldge_type == 1:
        source_buldge = al.lp.Sersic(
            centre=source_centre,
            ell_comps=source_ell_comps,
            intensity=source_intensity,
            effective_radius=max(random.random(), MIN_RADIUS),
            sersic_index=random.randint(1, 10)
        )
    elif source_buldge_type == 2:
        r1 = max(random.random(), MIN_RADIUS)
        r2 = max(random.random(), MIN_RADIUS)
        source_buldge = al.lp.Chameleon(
            centre=source_centre,
            ell_comps=source_ell_comps,
            intensity=source_intensity,
            core_radius_0=min(r1, r2),
            core_radius_1=max(r1,r2)
        )
    else:
        source_buldge = al.lp.Exponential(
            centre=source_centre,
            ell_comps=source_ell_comps,
            intensity=source_intensity,
            effective_radius=max(random.random(), MIN_RADIUS)
        )
    
    # DECIDING THE SOURCE'S DISK
    source_disk_type = random.randint(0,3)
    source_intensity = max(random.random(), MIN_SOURCE_INTENSITY)
    source_disk = None
    if source_disk_type == 0:
        source_disk = al.lp.Gaussian(
            centre=source_centre,
            ell_comps=source_ell_comps,
            intensity=source_intensity,
            sigma=max(random.random(),MIN_RADIUS)
        )
    elif source_disk_type == 1:
        source_disk = al.lp.Sersic(
            centre=source_centre,
            ell_comps=source_ell_comps,
            intensity=source_intensity,
            effective_radius=max(random.random(),MIN_RADIUS),
            sersic_index=random.randint(1, 10)
        )
    elif source_disk_type == 2:
        r1 = max(random.random(), MIN_RADIUS)
        r2 = max(random.random(), MIN_RADIUS)
        source_disk = al.lp.Chameleon(
            centre=source_centre,
            ell_comps=source_ell_comps,
            intensity=source_intensity,
            core_radius_0=min(r1, r2),
            core_radius_1=max(r1,r2)
        )
    else:
        source_disk = al.lp.Exponential(
            centre=source_centre,
            ell_comps=source_ell_comps,
            intensity=source_intensity,
            effective_radius=max(random.random(), MIN_RADIUS)
        )
    
    # ACTUAL SOURCE CREATION
    r1 = random.random()
    r2 = random.random()
        # Redshift of the source is the max of r1 and r2, that of the lensing galaxy is the min
    source = al.Galaxy(
        redshift=max(r1, r2),
        buldge = source_buldge,
        disk = source_disk
    )

    # ----CREATING THE LENSER GALAXY------
    lenser_center = (random.random()*HEIGHT* PIXEL_SCAlES/2, random.random()*WIDTH* PIXEL_SCAlES/2)

    MIN_LENSER_INTENSITY = source_intensity / 2
    
    # Deciding the normal matter mass profile:
    normal_matter_mp_type = random.randint(0,3)
    nm_center = (random.random()*HEIGHT* PIXEL_SCAlES/2, random.random()*WIDTH* PIXEL_SCAlES/2)
    nm_ell_comps = generate_ell_comps()
    nm_intensity = max(random.random(), MIN_LENSER_INTENSITY)
    nm_m = None
    nm_l = None

    if normal_matter_mp_type == 0:
        sigma=max(random.random(), MIN_RADIUS)
        nm_m = al.mp.Gaussian(
            centre=nm_center,
            ell_comps=nm_ell_comps,
            intensity=nm_intensity,
            sigma=sigma
        )
        nm_l = al.lp.Gaussian(
            centre=nm_center,
            ell_comps=nm_ell_comps,
            intensity=nm_intensity,
            sigma=sigma
        )
    elif normal_matter_mp_type == 1:
        eff_radius = max(random.random(), MIN_RADIUS)
        sersic_index = random.randint(1,10)
        nm_m = al.mp.Sersic(
            centre=nm_center,
            ell_comps=nm_ell_comps,
            intensity=nm_intensity,
            effective_radius=eff_radius,
            sersic_index=sersic_index
        )

        nm_l = al.lp.Sersic(
            centre=nm_center,
            ell_comps=nm_ell_comps,
            intensity=nm_intensity,
            effective_radius=eff_radius,
            sersic_index=sersic_index
        )
    elif normal_matter_mp_type == 2:
        rr1 = max(random.random(), MIN_RADIUS)
        rr2 = max(random.random(), MIN_RADIUS)
        nm_m = al.mp.Chameleon(
            centre=nm_center,
            ell_comps=nm_ell_comps,
            intensity=nm_intensity,
            core_radius_0=min(rr1, rr1),
            core_radius_1=max(rr1, rr2)
        )
        nm_l = al.lp.Chameleon(
            centre=nm_center,
            ell_comps=nm_ell_comps,
            intensity=nm_intensity,
            core_radius_0=min(rr1, rr1),
            core_radius_1=max(rr1, rr2)
        )
    elif normal_matter_mp_type == 3:
        eff_radius = max(random.random(), MIN_RADIUS)
        nm_m = al.mp.Exponential(
            centre=nm_center,
            ell_comps=nm_ell_comps,
            intensity=nm_intensity,
            effective_radius=eff_radius
        )
        nm_l = al.lp.Exponential(
            centre=nm_center,
            ell_comps=nm_ell_comps,
            intensity=nm_intensity,
            effective_radius=eff_radius
        )
    
    # Deciding the dark matter mass profile:
    dm_type = random.randint(0,7)
    dm_center = nm_center
    dm_ell_comps = generate_ell_comps()
    dm_intensity = max(random.random(), MIN_LENSER_INTENSITY)
    dm_mp = None
    if dm_type == 1:
        r1 = max(random.random(), MIN_RADIUS)
        r2 = max(random.random(), MIN_RADIUS)
        dm_mp = al.mp.Chameleon(
            centre=dm_center,
            ell_comps=dm_ell_comps,
            intensity=dm_intensity,
            core_radius_0=min(r1, r2),
            core_radius_1=max(r1,r2)
        )
    elif dm_type == 2:
        dm_mp = al.mp.DevVaucouleurs(
            centre=dm_center,
            ell_comps=dm_ell_comps,
            intensity=dm_intensity,
            effective_radius=max(random.random(), MIN_RADIUS)
        )
    elif dm_type == 3:
        dm_mp = al.mp.Exponential(
            centre=dm_center,
            ell_comps=dm_ell_comps,
            intensity=dm_intensity,
            effective_radius=max(random.random(), MIN_RADIUS)
        )
    elif dm_type == 4:
        dm_mp = al.mp.Gaussian(
            centre=dm_center,
            ell_comps=dm_ell_comps,
            intensity=dm_intensity,
            sigma=max(random.random(), MIN_RADIUS)
        )
    elif dm_type == 5:
        dm_mp = al.mp.gNFW(
            centre=dm_center,
            ell_comps=dm_ell_comps,
            kappa_s=random.random(),
            scale_radius=max(random.random(), MIN_RADIUS)
        )
    elif dm_type == 6:
        dm_mp = al.mp.Isothermal(
            centre=dm_center,
            ell_comps=dm_ell_comps,
            einstein_radius=max(random.random(), MIN_RADIUS)
        )
    elif dm_type == 7:
        dm_mp = al.mp.Sersic(
            centre=dm_center,
            ell_comps=dm_ell_comps,
            intensity=dm_intensity,
            effective_radius=max(random.random(), MIN_RADIUS),
            sersic_index=random.randint(1,10)
        )

    # Actually creating the lenser galaxy
    lenser = al.Galaxy(
        redshift=min(r1, r2),
        nm = nm_m,
        nm_l = nm_l,
        dm_m = dm_mp
    )

    toc = time.time()
    
    # tracer = al.Tracer.from_galaxies(galaxies=[source])
    # traced_image = tracer.image_2d_from(grid=grid)
    # plt.imshow(traced_image.native)
    # plt.title("Source galaxy")
    # plt.colorbar()
    # plt.show()
    
    # tracer = al.Tracer.from_galaxies(galaxies=[lenser])
    # traced_image = tracer.image_2d_from(grid=grid)
    # plt.imshow(traced_image.native)
    # plt.title("Lenser Galaxy")
    # plt.colorbar()
    # plt.show()
    
    tracer = al.Tracer.from_galaxies(galaxies=[source, lenser])
    traced_image = tracer.image_2d_from(grid=grid)
    # plt.imshow(traced_image.native)
    # plt.title("Lensed image")
    # plt.colorbar()
    # plt.show()

    # print("Simulated specs:", normal_matter_mp_type, source_buldge_type, source_disk_type)
    # print("Centers:", nm_center, source_centre)
    # print("Dark matter type:", dm_type)

    max_val = np.max(traced_image.native)

    tr = np.zeros((HEIGHT, WIDTH, 3), dtype=np.uint8)
    tr[:, :, 0] = traced_image.native * 255 / max_val
    tr[:, :, 1] = traced_image.native * 255 / max_val
    tr[:, :, 2] = traced_image.native * 255 / max_val

    img = Image.fromarray(tr, 'RGB')

    folder = 'data/' + CLASS_NAMES[dm_type]
    if not os.path.exists(folder):
        os.mkdir(folder)
    file_name = find_empty_file_starting_with(folder + '/s') #+ str(source_buldge_type) + '_' + str(source_disk_type) + '_' + str(normal_matter_mp_type))
    img.save(file_name)

    # print("Created source in", toc-tic, "secs")

In [ ]:
counts = [0 for i in range(5)]
for i in range(10000):
    counts[random.randint(0,4)] += 1
print(counts)